# Pydantic

Pydantic to biblioteka w Pythonie, która jest używana do konwersji, walidacji i serializacji danych. Podstawowym sposobem jej użycia jest zdefiniowanie modeli czyli klas dziedziczących po `BaseModel`, których struktura definiuje oczekiwany schemat danych:

In [2]:
from pydantic import BaseModel, model_validator
from datetime import datetime
from typing import Self

class OHLC(BaseModel):
    date: datetime
    open: float
    high: float
    low: float
    close: float
    
    @model_validator(mode="after")
    def validate_start_before_end(self: Self) -> Self:
        if self.close < self.low or self.open < self.low:
            raise ValueError("low must be the lowest value")
        if self.close > self.high or self.open > self.high:
            raise ValueError("high must be the highest value")
        return self

# dane odebrane np.: w postaci zapytania HTTP
raw_data = {
    "date": "2023-10-03 20:00:00",
    "open": 110.0,
    "high": 105.0,
    "low": 99.0,
    "close": 103.5
}

# Konwersja na obiekt klasy OHLC powoduje walidację i ew. rzucenie wyjątkiem ValueError
try:
    ohlc_entry = OHLC(**raw_data)
    print("Data is correct!")
except ValueError as e:
    print(f"Validation error: {e}")

Validation error: 1 validation error for OHLC
  Value error, high must be the highest value [type=value_error, input_value={'date': '2023-10-03 20:0...': 99.0, 'close': 103.5}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


W powyższym przykładzie klasa OHLC jest modelem i definiuje 2 metody walidujące określone aspekty danych. Następnie pod koniec przykładu konwertujemy słownik `raw_data` na obiekt OHLC, uruchamiając logikę walidującą. Pydantic bazuje na metodach oznaczonych jako `@model_validator` lub `@field_validator` oraz typach danych pól zdefiniowanych w modelu.

## Konwersja typów
Jeśli to możliwe, przekazane dane zostają automatycznie skonwertowane na typy zadeklarowane przy użyciu anotacji typowych:

In [ ]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

p = Person(name="Alan", age="25")
print(f"age is of type {type(p.age)} and has value {p.age}")

In [ ]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

try:
    p = Person(name="Alan", age="dwadzieścia pięć")
    print(f"age is of type {type(p.age)} and has value {p.age}")
except ValueError as e:
    print(e.errors())

Automatycznie dostajemy możliwość konwersji obiektów danego modelu na słowniki:

In [ ]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

    
p = Person(name="Alan", age=25)
p.model_dump()

... lub prosto do JSON-owego stringa:

In [ ]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

    
p = Person(name="Alan", age=25)
p.model_dump_json()

Możliwe jest także tworzenie instancji modelu bezpośrednio z JSON-a:

In [ ]:
Person.model_validate_json('{"name":"Alan","age":25}')

... lub słownika:

In [ ]:
Person.model_validate({'name': 'Adam', 'age': 23})

## Strict mode
W trybie *strict* nie następuje standardowa konwersja typów - dla obiektów Pythona wymagana jest pełna zgodność typowa. Wyjątkiem jest konwersja z JSON-a, gdzie konieczne jest dopuszczanie pewnej niezgodności (np. UUID jako str).

In [ ]:
from pydantic import BaseModel, ValidationError
class Person(BaseModel):
    name: str
    age: int

print(Person.model_validate({'name': 'Adrian', 'age': '123'}))  # lax mode
#> x=123

try:
    Person.model_validate({'name': 'Adrian', 'age': '123'}, strict=True)  # strict mode
except ValidationError as exc:
    print(exc)

Tryb "strict" można włączać dla całego modelu (np.: przez `config_dict`), dla pojedynczych pól czy podczas wołania `model_validate`

## Wartości domyślne i pola
Modele Pydantica mogą mieć wartości domyślne:

In [ ]:
class Student(BaseModel):
    name: str
    gpa: decimal.Decimal = decimal.Decimal("4.5")

... jednak muszą być one stałymi, znanymi w momencie definicji pola. Wynika to z automatycznej konwersji tak zadeklarowanych pól na parametry konstruktora. Jak wiadomo, parametry domyślne funkcji są wyliczane jedynie raz, gdy moduł definiujący funkcje jest po raz pierwszy ładowany. W przypadku wartości wyliczanych dynamicznie skutkowałoby to dość zaskakującym zachowaniem. Aby jednak osiągnąć ten sam efekt, możemy posłużyć się mechanizmem tak zwanych pól (ang. *fields*):

In [ ]:
import time
import uuid
from decimal import Decimal
from typing import Annotated
from pydantic import BaseModel, Field, AliasPath, AliasChoices


def fetch_gpa():
    # udajemy, że pobieramy dane z internetu:
    time.sleep(1)
    print("fetching....")
    return Decimal('4.5')

class Student(BaseModel):
    id: Annotated[str, Field(default_factory=lambda: uuid.uuid4().hex)]
    first_name: str = Field(
        validation_alias=AliasPath('name', 0),
        pattern=r'^[A-Z]\w+$'
    )
    last_name: str = Field(
        validation_alias=AliasPath('name', 1),
        pattern=r'^[A-Z]\w+$'
    )
    gpa: Decimal = Field(
        default_factory=fetch_gpa,
        alias=AliasChoices('srednia', 'gpa'),
        ge=Decimal('2.0'),
        le=Decimal('5.0'),
        repr=False
    )

janek = Student(name=["Janek", "Kowalski"])
print(repr(janek))

zosia = Student(name=["Zosia", "Kowalska"], gpa=Decimal('4.8'))
print(repr(zosia))

hania = Student(name=["Hania", "Nowak"], srednia='4.75')
print(repr(hania))

*Fields* mogą służyć nie tylko do dynamicznego generowania wartości domyślnych pól, ale również definiować ich aliasy, walidować wartości numeryczne czy zgodność z wyrażeniem regularnym.

## Generowanie schemy JSON
Pydantic umożliwia proste generowanie schemy JSON (ang. *jsonschema*) z modeli lub tak naprawdę dowolnych typów:

In [ ]:
import time
import uuid
import json
from decimal import Decimal
from typing import Annotated
from pydantic import BaseModel, Field

class Student(BaseModel):
    id: Annotated[str, Field(default_factory=lambda: uuid.uuid4().hex)]
    name: str
    gpa: Decimal

print(json.dumps(Student.model_json_schema(), indent=2))

In [ ]:
from pydantic import TypeAdapter

adapter = TypeAdapter(list[int])
adapter.json_schema()

## Niezmienialność (*Immutability*)
Podobnie jak `dataclasses`, modele Pydanticowe mogą być niezmienialne - służy do tego parametr `frozen=True|False`, który może działać na poziomie pól:

In [56]:
from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    name: str = Field(frozen=True)
    age: int


user = User(name='John', age=42)

try:
    user.name = 'Jane'  
except ValidationError as e:
    print(e)

1 validation error for User
name
  Field is frozen [type=frozen_field, input_value='Jane', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/frozen_field


Tak stworzone obiekty nie mogą mieć nadpisywanych pól oznaczonych jako "frozen". Aby stworzyć ich kopię ze zmienioną wartością należy użyć `model_copy`:

In [57]:
from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    name: str = Field(frozen=True)
    age: int


user = User(name='John', age=42)

try:
    user2 = user.model_copy(update={'name': 'Ian'})
    print(user2)
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    name
      Field is frozen [type=frozen_field, input_value='Jane', input_type=str]
    """

name='Ian' age=42


## Wydajność
https://docs.pydantic.dev/latest/concepts/performance/

## *Zadanie*
```
git checkout task-date-range
git checkout -b date-range-solution
pip-sync
```
W wielu miejscach kodu `StockTradera` używamy zakresu dat reprezentowanego przez obiekty klasy `DateRange`. Zmodyfikuj tę klasę z użyciem Pydantic tak, by:
- nie możliwe było stworzenie jej obiektów gdy start jest później niż koniec (start < end)
- nie możliwe było stworzenie jej obiektów metodami fabrycznymi years|months|days|hours_back, gdy podamy ujemne parametry
- metoda `from_last` akceptowała tylko napisy, w postaci, która jest parsowalna bez zmiany treści tej metody
- po utworzeniu nie było możliwe zmienianie wartości jej pól

## *Zadanie*
```
git checkout task-alpha-vantage
git checkout -b alpha-vantage-task-solution
```
W implementacji `DataSource`, która integruje się z Alpha Vantage dane są pobierane zapytaniem HTTP w postaci JSONa. Użyj Pydantica do jego zwalidowania zanim utworzymy z niego nowy `DataFrame`. W pliku `src/stock_trader/acquisition/data_sources/test_data/example_alpha_vantage_response.json` znajduje się przykładowy json przysłany w odpowiedzi z serwera.
- utwórz odpowiedni model Pydantica i przetestuj jego działanie na przykładowej odpowiedzi w powyższym pliku
- zintegruj swój kod z `AlphaVantageDataSource` - dane z odpowiedzi HTTP są przetwarzane w metodzie `_fetch_raw_data`
- jeśli masz konto na Alpha Vantage użyj swojego tokena i odpal kilkukrotnie stock_tradera podając token w parametrze `--alpha-vantage-api-key`
    - jak zachowuje się aplikacja gdy zapytanie trafia na rate limit?
    - (opcjonalnie) zaimplementuj ponawianie zapytania w przypadku błędu. Darmowy klucz Alpha Vantage umożliwia 5 zapytań na minutę.